In [4]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB5 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"sixrule.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "sixrule.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("nomorspt.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_episode: 0, score : -3986.0, n_buffer : 340, eps : 8.0%
--------------------------------------------------
flow time: 24232, util : 0.929, makespan : 751
n_episode: 1, score : -4055.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_episode: 1, score : -3986.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 24381, util : 0.922, makespan : 720
n_episode: 2, score : -4130.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 23783, util : 0.947, makespan : 719
n_episode: 2, score : -4019.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 24807, util : 0.928, makespan : 734
n_episode: 3, score : -4186.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time:

--------------------------------------------------
flow time: 27414, util : 0.914, makespan : 694
n_episode: 26, score : -4091.0, n_buffer : 9180, eps : 7.7%
--------------------------------------------------
flow time: 27317, util : 0.907, makespan : 704
n_episode: 27, score : -4192.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 24582, util : 0.931, makespan : 718
n_episode: 27, score : -4136.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 25503, util : 0.927, makespan : 719
n_episode: 28, score : -4180.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 26529, util : 0.923, makespan : 666
n_episode: 28, score : -3901.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 29196, util : 0.900, makespan : 696
n_episode: 29, score : -4093.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------


--------------------------------------------------
flow time: 27060, util : 0.896, makespan : 680
n_episode: 52, score : -3979.0, n_buffer : 18020, eps : 7.5%
--------------------------------------------------
flow time: 28197, util : 0.895, makespan : 683
n_episode: 53, score : -4016.0, n_buffer : 18360, eps : 7.5%
--------------------------------------------------
flow time: 33011, util : 0.902, makespan : 655
n_episode: 53, score : -3866.0, n_buffer : 18360, eps : 7.5%
--------------------------------------------------
flow time: 32746, util : 0.891, makespan : 675
n_episode: 54, score : -3980.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_episode: 54, score : -3986.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 24136, util : 0.935, makespan : 734
n_episode: 55, score : -4104.0, n_buffer : 19040, eps : 7.4%
----------------------------------------------

--------------------------------------------------
flow time: 25751, util : 0.931, makespan : 743
n_episode: 78, score : -4029.0, n_buffer : 26860, eps : 7.2%
--------------------------------------------------
flow time: 24218, util : 0.923, makespan : 737
n_episode: 79, score : -4135.0, n_buffer : 27200, eps : 7.2%
--------------------------------------------------
flow time: 30835, util : 0.895, makespan : 664
n_episode: 79, score : -3934.0, n_buffer : 27200, eps : 7.2%
--------------------------------------------------
flow time: 30951, util : 0.885, makespan : 682
n_episode: 80, score : -4024.0, n_buffer : 27540, eps : 7.2%
--------------------------------------------------
flow time: 30623, util : 0.891, makespan : 662
n_episode: 80, score : -3914.0, n_buffer : 27540, eps : 7.2%
--------------------------------------------------
flow time: 31723, util : 0.892, makespan : 674
n_episode: 81, score : -3992.0, n_buffer : 27880, eps : 7.2%
----------------------------------------------

--------------------------------------------------
flow time: 30986, util : 0.902, makespan : 669
n_episode: 104, score : -3970.0, n_buffer : 35700, eps : 7.0%
--------------------------------------------------
flow time: 26279, util : 0.918, makespan : 751
n_episode: 105, score : -4041.0, n_buffer : 36040, eps : 7.0%
--------------------------------------------------
flow time: 31482, util : 0.898, makespan : 656
n_episode: 105, score : -3885.0, n_buffer : 36040, eps : 7.0%
--------------------------------------------------
flow time: 29075, util : 0.892, makespan : 698
n_episode: 106, score : -4074.0, n_buffer : 36380, eps : 6.9%
--------------------------------------------------
flow time: 24023, util : 0.933, makespan : 734
n_episode: 106, score : -4057.0, n_buffer : 36380, eps : 6.9%
--------------------------------------------------
flow time: 25007, util : 0.916, makespan : 737
n_episode: 107, score : -4086.0, n_buffer : 36720, eps : 6.9%
----------------------------------------

...

--------------------------------------------------
flow time: 25699, util : 0.938, makespan : 795
n_episode: 924, score : -4533.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 26317, util : 0.938, makespan : 800
n_episode: 924, score : -4595.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 25640, util : 0.936, makespan : 786
n_episode: 925, score : -4464.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24027, util : 0.924, makespan : 734
n_episode: 925, score : -4236.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24526, util : 0.925, makespan : 743
n_episode: 926, score : -4166.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23514, util : 0.925, makespan : 716
n_episode: 926, score : -4015.0, n_buffer : 100000, eps : 1.0%
----------------------------------

--------------------------------------------------
flow time: 23981, util : 0.941, makespan : 721
n_episode: 949, score : -3997.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24009, util : 0.939, makespan : 732
n_episode: 950, score : -4024.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24532, util : 0.916, makespan : 761
n_episode: 950, score : -4204.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24372, util : 0.912, makespan : 727
n_episode: 951, score : -4096.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 26592, util : 0.904, makespan : 671
n_episode: 951, score : -3955.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 27522, util : 0.905, makespan : 668
n_episode: 952, score : -3922.0, n_buffer : 100000, eps : 1.0%
----------------------------------

--------------------------------------------------
flow time: 26108, util : 0.910, makespan : 664
n_episode: 975, score : -3891.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23211, util : 0.932, makespan : 720
n_episode: 975, score : -3966.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23225, util : 0.928, makespan : 736
n_episode: 976, score : -3997.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23139, util : 0.939, makespan : 738
n_episode: 976, score : -4003.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 23229, util : 0.934, makespan : 723
n_episode: 977, score : -4057.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24788, util : 0.929, makespan : 664
n_episode: 977, score : -3945.0, n_buffer : 100000, eps : 1.0%
----------------------------------

FlowTime: 25297
machine_util: 0
util: 0.9132933533233384
makespan: 682
Score -4002.0


RuntimeError: Error(s) in loading state_dict for Qnet:
	size mismatch for fc3.weight: copying a param with shape torch.Size([9, 32]) from checkpoint, the shape in current model is torch.Size([6, 32]).
	size mismatch for fc3.bias: copying a param with shape torch.Size([9]) from checkpoint, the shape in current model is torch.Size([6]).

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB5 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,6)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 5)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"sixrule.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "sixrule.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("nomorspt.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 24874, util : 0.906, makespan : 747
n_episode: 0, score : -8854.0, n_buffer : 340, eps : 8.0%
--------------------------------------------------
flow time: 25370, util : 0.901, makespan : 728
n_episode: 1, score : -8577.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 24874, util : 0.906, makespan : 747
n_episode: 1, score : -8854.0, n_buffer : 680, eps : 8.0%


<ipython-input-1-b99897cbe271>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 25575, util : 0.883, makespan : 752
n_episode: 2, score : -8897.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 27517, util : 0.945, makespan : 817
n_episode: 2, score : -9783.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 27349, util : 0.939, makespan : 851
n_episode: 3, score : -10333.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 25645, util : 0.958, makespan : 838
n_episode: 3, score : -10227.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 26178, util : 0.948, makespan : 886
n_episode: 4, score : -10417.0, n_buffer : 1700, eps : 8.0%
--------------------------------------------------
flow time: 26146, util : 0.963, makespan : 859
n_episode: 4, score : -10979.0, n_buffer : 1700, eps : 8.0%
--------------------------------------------------
flo

--------------------------------------------------
flow time: 35167, util : 0.879, makespan : 700
n_episode: 28, score : -8362.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 36876, util : 0.889, makespan : 712
n_episode: 28, score : -8569.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 34973, util : 0.884, makespan : 724
n_episode: 29, score : -8625.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------
flow time: 31984, util : 0.902, makespan : 828
n_episode: 29, score : -9746.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------
flow time: 29610, util : 0.907, makespan : 821
n_episode: 30, score : -9802.0, n_buffer : 10540, eps : 7.7%
--------------------------------------------------
flow time: 29974, util : 0.914, makespan : 804
n_episode: 30, score : -9590.0, n_buffer : 10540, eps : 7.7%
------------------------------------------------

--------------------------------------------------
flow time: 26032, util : 0.933, makespan : 849
n_episode: 54, score : -9894.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 25237, util : 0.955, makespan : 808
n_episode: 54, score : -9789.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 25715, util : 0.937, makespan : 827
n_episode: 55, score : -10253.0, n_buffer : 19040, eps : 7.4%
--------------------------------------------------
flow time: 28181, util : 0.924, makespan : 836
n_episode: 55, score : -10317.0, n_buffer : 19040, eps : 7.4%
--------------------------------------------------
flow time: 28452, util : 0.927, makespan : 827
n_episode: 56, score : -9936.0, n_buffer : 19380, eps : 7.4%
--------------------------------------------------
flow time: 29110, util : 0.929, makespan : 804
n_episode: 56, score : -9607.0, n_buffer : 19380, eps : 7.4%
--------------------------------------------

...

--------------------------------------------------
flow time: 25017, util : 0.906, makespan : 687
n_episode: 568, score : -8098.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24964, util : 0.897, makespan : 713
n_episode: 569, score : -8241.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24401, util : 0.912, makespan : 733
n_episode: 569, score : -8298.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24284, util : 0.923, makespan : 739
n_episode: 570, score : -8683.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 25722, util : 0.920, makespan : 667
n_episode: 570, score : -7843.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24687, util : 0.922, makespan : 679
n_episode: 571, score : -8016.0, n_buffer : 100000, eps : 2.3%
----------------------------------

--------------------------------------------------
flow time: 24449, util : 0.911, makespan : 699
n_episode: 594, score : -8187.0, n_buffer : 100000, eps : 2.1%
--------------------------------------------------
flow time: 24505, util : 0.908, makespan : 757
n_episode: 594, score : -8360.0, n_buffer : 100000, eps : 2.1%
--------------------------------------------------
flow time: 25023, util : 0.902, makespan : 759
n_episode: 595, score : -8689.0, n_buffer : 100000, eps : 2.0%
--------------------------------------------------
flow time: 24838, util : 0.907, makespan : 682
n_episode: 595, score : -8079.0, n_buffer : 100000, eps : 2.0%
--------------------------------------------------
flow time: 24981, util : 0.907, makespan : 693
n_episode: 596, score : -8260.0, n_buffer : 100000, eps : 2.0%
--------------------------------------------------
flow time: 25225, util : 0.924, makespan : 665
n_episode: 596, score : -7902.0, n_buffer : 100000, eps : 2.0%
----------------------------------

--------------------------------------------------
flow time: 25047, util : 0.903, makespan : 691
n_episode: 619, score : -8091.0, n_buffer : 100000, eps : 1.8%
--------------------------------------------------
flow time: 25154, util : 0.901, makespan : 688
n_episode: 620, score : -8142.0, n_buffer : 100000, eps : 1.8%
--------------------------------------------------
flow time: 24186, util : 0.922, makespan : 744
n_episode: 620, score : -8454.0, n_buffer : 100000, eps : 1.8%
--------------------------------------------------
flow time: 24506, util : 0.910, makespan : 752
n_episode: 621, score : -8873.0, n_buffer : 100000, eps : 1.8%
--------------------------------------------------
flow time: 25004, util : 0.906, makespan : 677
n_episode: 621, score : -8032.0, n_buffer : 100000, eps : 1.8%
--------------------------------------------------
flow time: 24482, util : 0.922, makespan : 697
n_episode: 622, score : -8097.0, n_buffer : 100000, eps : 1.8%
----------------------------------

--------------------------------------------------
flow time: 24724, util : 0.910, makespan : 722
n_episode: 645, score : -8616.0, n_buffer : 100000, eps : 1.6%
--------------------------------------------------
flow time: 24790, util : 0.904, makespan : 750
n_episode: 645, score : -8416.0, n_buffer : 100000, eps : 1.6%
--------------------------------------------------
flow time: 24805, util : 0.907, makespan : 737
n_episode: 646, score : -8417.0, n_buffer : 100000, eps : 1.5%
--------------------------------------------------
flow time: 24818, util : 0.907, makespan : 732
n_episode: 646, score : -8638.0, n_buffer : 100000, eps : 1.5%
--------------------------------------------------
flow time: 24915, util : 0.902, makespan : 736
n_episode: 647, score : -8625.0, n_buffer : 100000, eps : 1.5%
--------------------------------------------------
flow time: 24325, util : 0.931, makespan : 722
n_episode: 647, score : -8180.0, n_buffer : 100000, eps : 1.5%
----------------------------------

--------------------------------------------------
flow time: 24736, util : 0.911, makespan : 716
n_episode: 670, score : -8124.0, n_buffer : 100000, eps : 1.3%
--------------------------------------------------
flow time: 24858, util : 0.908, makespan : 680
n_episode: 671, score : -7986.0, n_buffer : 100000, eps : 1.3%
--------------------------------------------------
flow time: 26049, util : 0.903, makespan : 672
n_episode: 671, score : -7914.0, n_buffer : 100000, eps : 1.3%
--------------------------------------------------
flow time: 26518, util : 0.908, makespan : 671
n_episode: 672, score : -7964.0, n_buffer : 100000, eps : 1.3%
--------------------------------------------------
flow time: 29563, util : 0.897, makespan : 669
n_episode: 672, score : -7910.0, n_buffer : 100000, eps : 1.3%
--------------------------------------------------
flow time: 29563, util : 0.897, makespan : 669
n_episode: 673, score : -7910.0, n_buffer : 100000, eps : 1.3%
----------------------------------